In [71]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

In [83]:
data_types = ['Wake','N1', 'N2', 'N3', 'REM']

load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
X = pd.read_hdf(load_path + 'N2six_second_freq_df.h5', key='df', mode='r') 
y = pd.read_hdf(load_path + 'N2_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'N2_groups.h5', key='df', mode='r') 
X_subset = X.copy()
# Time series columns are -->  list(X_subset.columns)
# X_subset.head()

#Convert X to a TS Fresh format dataframe --> ts_fresh_df
t1 = time.time()
# X_subset = X_subset.iloc[: , :7]
ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X_subset, ts_cols = list(X_subset.columns))
t2 = time.time()
print(t2-t1)

0.45073986053466797


In [84]:
t1 = time.time()

settings = MinimalFCParameters()
# settings = EfficientFCParameters()
extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=settings)

t2 = time.time()
t2-t1

24-Aug-23 13:52:03 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 56.84it/s]


3.128920555114746

In [85]:
extracted_ts_fresh_df

,Delta_Central__sum_values,Delta_Central__median,Delta_Central__mean,Delta_Central__length,Delta_Central__standard_deviation,Delta_Central__variance,Delta_Central__root_mean_square,Delta_Central__maximum,Delta_Central__absolute_maximum,Delta_Central__minimum,...,TotalAbsPow_Right Temporal__sum_values,TotalAbsPow_Right Temporal__median,TotalAbsPow_Right Temporal__mean,TotalAbsPow_Right Temporal__length,TotalAbsPow_Right Temporal__standard_deviation,TotalAbsPow_Right Temporal__variance,TotalAbsPow_Right Temporal__root_mean_square,TotalAbsPow_Right Temporal__maximum,TotalAbsPow_Right Temporal__absolute_maximum,TotalAbsPow_Right Temporal__minimum
0,93.463389,0.574442,0.566445,165.0,0.123394,0.015226,0.579729,0.817506,0.817506,0.153710,...,8064.270716,45.322511,48.874368,165.0,21.147596,447.220806,53.253400,148.789346,148.789346,17.175054
1,139.869470,0.595810,0.582789,240.0,0.124007,0.015378,0.595837,0.835477,0.835477,0.188488,...,10350.453457,39.340242,43.126889,240.0,19.813391,392.570454,47.460500,143.770786,143.770786,15.655833
2,1020.928565,0.717451,0.689817,1480.0,0.136002,0.018497,0.703096,0.943543,0.943543,0.156514,...,54786.820087,31.349027,37.018122,1480.0,22.388759,501.256530,43.261968,341.796936,341.796936,9.434184
3,1094.382194,0.590175,0.589963,1855.0,0.117678,0.013848,0.601585,0.926150,0.926150,0.203314,...,142851.186559,59.159247,77.008726,1855.0,59.038286,3485.519257,97.035370,970.764099,970.764099,17.576897
4,393.173034,0.610535,0.609571,645.0,0.110343,0.012175,0.619477,0.918036,0.918036,0.258283,...,51269.368058,63.463866,79.487392,645.0,55.530245,3083.608106,96.963156,552.597989,552.597989,20.557353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1089.345233,0.645080,0.640791,1700.0,0.142655,0.020350,0.656478,0.946294,0.946294,0.152757,...,78879.207277,35.962176,46.399534,1700.0,33.503844,1122.507574,57.231323,299.038847,299.038847,11.046253
67,1296.941125,0.781356,0.760669,1705.0,0.137544,0.018918,0.773005,0.976096,0.976096,0.205141,...,110880.775267,47.342932,65.032713,1705.0,54.863626,3010.017455,85.083907,496.261495,496.261495,10.415712
68,1259.266223,0.612204,0.608341,2070.0,0.134601,0.018117,0.623054,0.950340,0.950340,0.107117,...,93827.657853,37.188001,45.327371,2070.0,30.346494,920.909712,54.547963,472.353872,472.353872,11.622822
69,1147.329849,0.631506,0.625248,1835.0,0.124778,0.015570,0.637577,0.926791,0.926791,0.156523,...,89162.207706,40.152716,48.589759,1835.0,28.458230,809.870826,56.310172,251.117311,251.117311,13.467179


In [86]:
#Generate a region to features dictionary - this will enable us to run expts regionally as before
regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())
for region in regions:
    region_features = [col for col in extracted_ts_fresh_df.dropna(axis = 1).columns if region + '_' in col]
    if len(region_features) > 0 : 
        regional_features_dict[region] = region_features

In [87]:
t1 = time.time()

#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(extracted_ts_fresh_df.dropna(axis = 1),y,groups, 2 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )

t2 = time.time()

t2-t1

11.757408142089844

In [88]:
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0)).T)

,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.347778,0.383333,0.301111,0.422222,0.473333,0.414444,0.501111,0.481111,0.566667,0.423333,0.476667,0.404444,0.511111
